# Creating the Testing Enviroment

For the testing environment, I have individually created 5 different user music listening history and I will begin my visualizing the embedding space features of the songs for each user.

In [1]:
import spotipy
import pandas as pd
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

#read cid from file
with open("ids/cid.txt", "r") as file:
    cid = file.read()

#read secret from file
with open("ids/secret.txt", "r") as file:
    secret = file.read()

#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

Collecting features from 5 pre-defined playlists that exhibit different types of music, and putting them into an embdding space for visualization.

In [2]:
session_1_uri = "spotify:playlist:4BbncuGqRY0GzR0t57PCKy"
session_2_uri = "spotify:playlist:1UMi6pQ4hqyDMbEhbshD9d"
session_3_uri = "spotify:playlist:55yqh8VajCF49DDohXO0nS"
session_4_uri = "spotify:playlist:28wpyiXjRFe8RoE65UHtWo"
session_5_uri = "spotify:playlist:1Bjl9TNi4gdi093qQzalmC"

sessions = [session_1_uri, session_2_uri, session_3_uri, session_4_uri, session_5_uri]

In [12]:
# create a pandas dataframe with the features of the songs in all the playlists
def get_playlist_features(playlist_URI):
    playlist_features_list = []
    playlist = sp.playlist(playlist_URI)
    for track in playlist["tracks"]["items"]:
        track_uri = track["track"]["uri"]
        playlist_features = sp.audio_features(track_uri)[0]

        playlist_features_list.append(playlist_features)

    playlist_features_df = pd.DataFrame(playlist_features_list)
    playlist_features_df.drop(["analysis_url", "track_href", "type", "uri", "id"], axis=1, inplace=True)


    return playlist_features_df


In [16]:
playlist_list = []
for i, playlist_URI in enumerate(sessions):
    features = get_playlist_features(playlist_URI)
    features["session"] = i
    playlist_list.append(features)


playlist_df = pd.concat(playlist_list)
playlist_df = pd.concat([playlist_df, pd.get_dummies(playlist_df["session"], prefix="session")], axis=1)
playlist_df.drop(["session"], axis=1, inplace=True)

In [17]:
playlist_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,session_0,session_1,session_2,session_3,session_4
0,0.645,0.498,7,-9.681,0,0.0376,0.811,0.529000,0.1010,0.331,111.963,193629,4,1,0,0,0,0
1,0.675,0.327,8,-13.501,0,0.0434,0.272,0.782000,0.1380,0.354,99.002,198826,4,1,0,0,0,0
2,0.605,0.687,9,-13.928,0,0.0450,0.479,0.949000,0.1040,0.120,106.896,345187,4,1,0,0,0,0
3,0.760,0.401,1,-9.537,0,0.0674,0.564,0.259000,0.0977,0.619,115.989,248892,4,1,0,0,0,0
4,0.865,0.384,10,-11.953,1,0.0577,0.024,0.434000,0.0682,0.852,122.480,219560,4,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,0.755,0.326,8,-7.311,0,0.0470,0.440,0.000005,0.1130,0.388,64.953,220000,4,0,0,0,0,1
44,0.436,0.358,9,-12.147,0,0.2830,0.850,0.059700,0.1900,0.333,82.135,305760,4,0,0,0,0,1
45,0.590,0.438,6,-12.594,0,0.0508,0.292,0.002160,0.1030,0.727,83.506,250867,4,0,0,0,0,1
46,0.622,0.409,5,-13.484,0,0.0362,0.650,0.023000,0.1420,0.785,86.173,260867,4,0,0,0,0,1


I am using UMAP dimensionality reduction to visualize the songs and identify clusters.